In [ ]:
def recommend(model, user_id, business_ids, top_n=10):
    """
    Recommend the top N businesses for a given user based on predicted ratings.
    
    Parameters:
    model: A model that takes (user_id, business_id) as input and predicts a rating between 1 and 5.
    user_id: The ID of the user for whom recommendations are being generated.
    business_ids: A list of business IDs to consider for recommendation.
    top_n: The number of top recommendations to return (default is 10).
    
    Returns:
    A sorted list of (business_id, predicted_rating) tuples, in descending order of predicted rating.
    """
    predictions = [(business_id, model.predict(user_id, business_id)) for business_id in business_ids]
    
    # Sort businesses by predicted rating in descending order and return the top_n
    recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)[:top_n]
    
    return recommendations


def evaluate(model, test_df, threshold=3, top_n=10):
    """
    Evaluate the recommendation model using precision and recall.
    
    Parameters:
    model: A model that predicts ratings for (user_id, business_id) pairs.
    test_df: A DataFrame containing columns ['user_id', 'business_id', 'rating'].
    threshold: The minimum rating to consider a business as relevant (default is 3).
    top_n: The number of top recommendations to consider per user (default is 10).
    
    Returns:
    A tuple (recall, precision) for the model's recommendations.
    """
    total_relevant = 0
    total_recommended = 0
    total_matched = 0
    
    users = test_df['user_id'].unique()
    
    for user_id in users:
        user_data = test_df[test_df['user_id'] == user_id]
        actual_relevant = set(user_data[user_data['rating'] >= threshold]['business_id'])
        
        recommended = set(business_id for business_id, _ in recommend(model, user_id, user_data['business_id'].tolist(), top_n))
        
        total_relevant += len(actual_relevant)
        total_recommended += len(recommended)
        total_matched += len(actual_relevant & recommended)
        #this is to watch the progressive metrics if th eevaluation is taking long.
        print(total_matched / total_relevant) if total_relevant > 0 else 0
        print(total_matched / total_recommended) if total_recommended > 0 else 0
    
    recall = total_matched / total_relevant if total_relevant > 0 else 0
    precision = total_matched / total_recommended if total_recommended > 0 else 0
    
    return recall, precision